<a href="https://colab.research.google.com/github/ZheRao/Python-Projects/blob/main/Bond_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# Getting the data ready

In [ ]:
# upload file from local drive to colab
from google.colab import files
uploaded = files.upload()

Saving DataClean2021_update_data.xlsx to DataClean2021_update_data.xlsx


In [ ]:
# read the bonds information, store it in "dataset" data frame
dataset = pd.read_excel('DataClean2021_update_data.xlsx')

In [ ]:
# read the yields information, store it in "yields" data frame
yields = pd.read_excel('DataClean2021_update_data.xlsx', sheet_name=1)


In [ ]:
# calculate the number of rows in yields
nrow_y = yields.iloc[:,1].count()

# calculate the number of columns in yields
ncol_y = yields.iloc[1,:].count()

# Calculating missing coupon numbers

In [ ]:
# merge the data so that I don't have to search the yields everytime I need to find a yield 
#  (reduce running time)
# Yield data starting in column 10 (NR), the first yield starts in 12 (025YR)
mergedData = pd.merge(left=dataset,right=yields,how="left", left_on="PurchaseDate", right_on="Date")

In [ ]:
# return the appropriate yields corresponding to the purchase date
def getYields2(bond):
  freq = bond.loc["PaymentPeriod"]
  #print(freq)
  if freq == "Annually":
    index = np.arange(15,10+ncol_y,4)
    return bond[index]
  elif freq == "SemiAnnually":
    index = np.arange(13,10+ncol_y,2)
    return bond[index]
  else:
    index = np.arange(12,10+ncol_y)
    return bond[index]

In [ ]:
# define the function that calculate the bond price
def bondPrice(period, coupon, spread, bondyield, num_coupon):
  price = 0

  if period == "Annually":
    for i in np.arange(num_coupon):
      price += coupon / (1+bondyield[i]+spread) ** (i+1)
    price += 100 / (1+bondyield[num_coupon-1]+spread)**(num_coupon)
  elif period == "SemiAnnually":
    for i in np.arange(num_coupon):
      price += coupon / (1+bondyield[i]/2+spread) ** (i+1)
    price += 100 / (1+bondyield[num_coupon-1]/2+spread)**(num_coupon)
  else:
    for i in np.arange(num_coupon):
      price += coupon / (1+bondyield[i]/4+spread) ** (i+1)
    price += 100 / (1+bondyield[num_coupon-1]/4+spread)**(num_coupon)
  
  return price

In [ ]:
# define the linear search function just in case bisection method cannot be used
def lineSearch(freq,coupon,spread,bondyield,TP,eps,max,min):
  price = bondPrice(freq,coupon,spread,bondyield,max)
  error = abs(price-TP)
  if error <= eps:
    return max
  lowest_error = error
  lowest_error_idx = max

  # return the number of coupons if the error is within tolerance
  for i in np.arange((max-min)):
    price = bondPrice(freq,coupon,spread,bondyield,max-i)
    error = abs(price-TP)
    if error <= eps:
      return max-i
    else:
      if error < lowest_error:
        lowest_error = error
        lowest_error_idx = max-i

  # if the function reaches here, means we have not returned anything, no error is within tolerance
  #  so we will return the coupon number that produce the least error
  return lowest_error_idx



In [ ]:
# define the function that calculate the difference between calculated price and true price
def diffFun(freq,coupon,spread,bondyield,couponNum,TP):
  price = bondPrice(freq,coupon,spread,bondyield,couponNum)
  return price-TP

In [ ]:
# define the bisection method

def getCouponNum2(freq,coupon,spread,bondyield,TP,min,eps):
  a = min
  b = bondyield.count()
  preDiff = 0
  curDiff = 0
  # first check if the signs are different in the two extreme cases
  f_a = diffFun(freq,coupon,spread,bondyield,a,TP)
  f_b = diffFun(freq,coupon,spread,bondyield,b,TP)
  
  if f_a*f_b >= 0:
    # pass the bond to linear search
    CNum=lineSearch(freq,coupon,spread,bondyield,TP,eps,b,min)
    return CNum


  # now that we know the root is in the middle, which means we can perform a bisection to find it
  c = round((a+b)/2)
  f_c = diffFun(freq,coupon,spread,bondyield,c,TP)

  while abs(f_c) > eps:
    if f_c * f_a < 0:
      b = c
      f_b = f_c
    elif f_c * f_b < 0:
      a = c
      f_a = f_c
    preDiff = f_c
    c = round((a+b)/2)
    f_c = diffFun(freq,coupon,spread,bondyield,c,TP)
    curDiff = f_c
    if preDiff == curDiff:
      CNum=lineSearch(freq,coupon,spread,bondyield,TP,eps,bondyield.count(),min)
      return CNum

  return c






In [ ]:
# calcualte the total number of coupons and store it in the bonds dataframe
dataset["TotalCouponNum"] = mergedData.apply(lambda x:getCouponNum2(x["PaymentPeriod"],x["CouponRate"]*100,
                                    x["Spread"],getYields2(x),
                                    x["PriceAtCoupon0(%)"],x["CurrentCouponNumbers"],0.0001),
                        axis=1)

# Calculating current bond prices

In [ ]:
# import the package to calculate dates
from datetime import date
from dateutil.relativedelta import relativedelta

In [ ]:
# define today's date as datetime.date object
today = date(2021, 8, 25)

In [ ]:
# define a function that return the number of months passed by for given number of coupon payments
def getMonths(bond):
  if bond["PaymentPeriod"] == "Annually":
    return 12 * bond["CurrentCouponNumbers"]

  elif bond["PaymentPeriod"] == "SemiAnnually":
    return 6 * bond["CurrentCouponNumbers"]

  else:
    return 3 * bond["CurrentCouponNumbers"]

In [ ]:
# find the nearest past coupon payment date, it returns a datetime.date object
def getCDate(bond):
  oriDate = bond["PurchaseDate"]
  curDate = date(oriDate.year, oriDate.month, oriDate.day)
  curDate += relativedelta(months =+ getMonths(bond))
  return curDate


In [ ]:
# find the number of days between last coupon date and today
def getDaysDiff(bond):
  delta = today - getCDate(bond)
  return delta.days

In [ ]:
# define a function to find desired yields based on date
def findY(today):  
  for i in np.arange(nrow_y):
    if yields.iloc[i,1].year == today.year and yields.iloc[i,1].month == today.month and yields.iloc[i,1].day == today.day:
      todayY = yields.iloc[i,2:ncol_y]
      return todayY


In [ ]:
# retrieve the yield curve on Aug 25, 2021
myYields = findY(today)

In [ ]:
# define a function that returns appropriate yields that should be used based on the coupon payment frequency
def getYields3(bond):
  freq = bond.loc["PaymentPeriod"]
  if freq == "Annually":
    index = np.arange(3,ncol_y-2,4)
    return myYields[index]
  elif freq == "SemiAnnually":
    index = np.arange(1,ncol_y-2,2)
    return myYields[index]
  else:
    index = np.arange(0,ncol_y-2)
    return myYields[index]

In [ ]:
# define a function that calculate the current bond price 
def bondPrice2(bond):
  freq = bond["PaymentPeriod"]
  spread = bond["Spread"]
  # calculate the bond price at the last coupon date
  price = bondPrice(freq,bond["CouponRate"]*100,spread,
                   getYields3(bond),(bond["TotalCouponNum"]-bond["CurrentCouponNumbers"]))
  
  rate = 0
  totalNum = 0
  diffNum = getDaysDiff(bond)

  # now find the bond price today
  if freq == "Annually":
    rate = myYields[3]
    totalNum = 360
    price *= (1+rate+spread) ** (diffNum/totalNum)
  elif freq == "SemiAnnually":
    rate = myYields[1]
    totalNum = 180
    price *= (1+rate/2+spread) ** (diffNum/totalNum)
  else:
    rate = myYields[0]
    totalNum = 90
    price *= (1+rate/4+spread) ** (diffNum/totalNum)
  

  return price
  

In [ ]:
# calcualte the curent prices and store it in the bonds dataframe
dataset["CurrentPrice"] = dataset.apply(lambda x:bondPrice2(x), axis=1)

# Calculate capital requirements of bond portfolio following Basel III

In [ ]:
# annualize the spread
def annualSpread(bond):
  spread = bond["Spread"]
  freq = bond["PaymentPeriod"]

  if freq == "Annually":
    return spread
  elif freq == "SemiAnnually":
    return spread*2
  else:
    return spread*4

In [ ]:
# agument annualized spread into data frame
dataset["AnnualizedSpread"] = dataset.apply(lambda x:annualSpread(x), axis=1)

In [ ]:
# get the spread decile and print the bounds
max = np.max(dataset["AnnualizedSpread"].values)
min = np.min(dataset["AnnualizedSpread"].values)
dataset["SpreadDecile"] = dataset.apply(lambda x: (x["AnnualizedSpread"]-min)/(max-min), axis=1)


In [ ]:
# print the bounds of spread
s005=0
sd005=0
s020=0
sd020=0
s040=0
sd040=0
s060=0
sd060=0
s075=0
sd075=0
s090=0
sd090=0

for i in np.arange(len(dataset)):
  spreadDecile = dataset["SpreadDecile"][i]
  if spreadDecile < 0.05:
    if spreadDecile > sd005:
      sd005 = spreadDecile
      s005 = dataset["AnnualizedSpread"][i]
  elif spreadDecile  < 0.2:
    if spreadDecile > sd020:
      sd020 = spreadDecile
      s020 = dataset["AnnualizedSpread"][i]
  elif spreadDecile  < 0.4:
    if spreadDecile > sd040:
      sd040 = spreadDecile
      s040 = dataset["AnnualizedSpread"][i]
  elif spreadDecile  < 0.6:
    if spreadDecile > sd060:
      sd060 = spreadDecile
      s060 = dataset["AnnualizedSpread"][i]
  elif spreadDecile  < 0.75:
    if spreadDecile > sd075:
      sd075 = spreadDecile
      s075 = dataset["AnnualizedSpread"][i]
  elif spreadDecile  < 0.9:
    if spreadDecile > sd090:
      sd090 = spreadDecile
      s090 = dataset["AnnualizedSpread"][i]

print("The 5%% cutoff: %0.3f%%\nThe 20%% cutoff: %0.3f%%\nThe 40%% cutoff: %0.3f%%\nThe 60%% cutoff: %0.3f%%\nThe 75%% cutoff: %0.3f%%\nThe 90%% cutoff: %0.3f%%\nThe max: %0.3f%%\n"
    %(s005*100,s020*100,s040*100,s060*100,s075*100,s090*100,max*100))


The 5% cutoff: 3.010%
The 20% cutoff: 12.008%
The 40% cutoff: 24.003%
The 60% cutoff: 36.000%
The 75% cutoff: 44.973%
The 90% cutoff: 54.000%
The max: 60.000%



In [ ]:
# define a function that returns the PD of a bond given its spread

def getPD(bond):
  spreadDecile = bond["SpreadDecile"]
  if spreadDecile < 0.05:
    return 0.001
  elif spreadDecile  < 0.2:
    return 0.01
  elif spreadDecile  < 0.4:
    return 0.12
  elif spreadDecile  < 0.6:
    return 0.36
  elif spreadDecile  < 0.75:
    return 0.52
  elif spreadDecile  < 0.9:
    return 0.75
  elif spreadDecile  <= 1:
    return 0.85
  else:
    return 1
  

In [ ]:
# agument the PD of the bonds into the dataset
dataset["PD"] = dataset.apply(lambda x: getPD(x), axis = 1)

In [ ]:
# define LGD
LGD = 0.75

In [ ]:
# agument the exposure at default to the bond dataset
dataset["EAD"] = dataset.apply(lambda x: x["FaceValue"]*x["CurrentPrice"]/100, axis=1)

In [ ]:
# define a function to return the maturity in years of a bond
def getYears(bond):
  freq = bond["PaymentPeriod"]
  totl = bond["TotalCouponNum"] - bond["CurrentCouponNumbers"]

  if freq == "Annually":
    return totl
  elif freq == "SemiAnnually":
    return totl/2
  else:
    return totl/4

In [ ]:
# augment the maturity in years to the bond dataset
dataset["MaturityInYears"] = dataset.apply(lambda x: getYears(x), axis=1)

## provision calculation

In [ ]:
# provision
dataset["Provision"] = dataset.apply(lambda x: x["PD"]*x["EAD"]*0.75, axis=1)

## capital calculation

In [ ]:
# calculating b
def getb(bond):
  myPD = bond["PD"]
  b = 0.11852 - 0.05478 * np.log(myPD)
  b = b**2
  return b

In [ ]:
# calculating ajusted maturity
def getM(bond):
  b = getb(bond)
  m = bond["MaturityInYears"]
  m = (1+(m-2.5)*b) / (1-1.5*b)
  return m

In [ ]:
# calculating R
def getR(bond):
  myPD = bond["PD"]
  R = 0.12 * ((1-np.exp(-50*myPD))/(1-np.exp(-50)))
  R += 0.24 * (1-((1-np.exp(-50*myPD))/(1-np.exp(-50))))
  return R


In [ ]:
# calculating the capital requirement
def capital(bond):
  myPD = bond["PD"]
  R = getR(bond)
  M = getM(bond)
  K = norm.cdf( np.sqrt(1/(1-R)) * norm.ppf(myPD) + np.sqrt(R/(1-R)) * norm.ppf(0.999) ) - myPD 
  K = 0.75 * M * K
  return K

In [ ]:
# augment the capital requirement (%) for each bond
dataset["CapitalRequirement%"] = dataset.apply(lambda x: capital(x), axis=1)

In [ ]:
# augment the capital requirement ($) for each bond
dataset["CapitalRequirement"] = dataset.apply(lambda x: capital(x)*x["EAD"], axis=1)

In [ ]:
# print total exposures
totalEAD = np.sum(dataset["EAD"].values)
totalEAD2 = "${:,.4f}".format(totalEAD)
print("Total value of the portfolio is ", totalEAD2)

# print total provision
totalPro = np.sum(dataset["Provision"].values)
totalPro2 = "${:,.4f}".format(totalPro)
print("Total Provisions required for the portfolio is ", totalPro2)

# print total capital requirement percentage
totalCapital = np.sum(dataset["CapitalRequirement"].values)
percentage = totalCapital/totalEAD
print("Total capital requirement percentage is %0.4f%%" %(percentage*100))

# print total capital requirement in $
totalCapital2 = "${:,.4f}".format(totalCapital)
print("Total capital requirement is ", totalCapital2)

Total value of the portfolio is  $13,271,024,823.2358
Total Provisions required for the portfolio is  $326,015,232.0340
Total capital requirement percentage is 24.5278%
Total capital requirement is  $3,255,086,381.9375
